### SImple Genai app using Langchain.

In [81]:
import os 
from dotenv import load_dotenv
load_dotenv()

api_key2 = os.getenv("OPENAI_API_KEY")
api_langchain = os.getenv("LANGCHAIN_API_KEY")

langchain_project = os.getenv("LANGCHAIN_PROJECT", "default_project")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = langchain_project

In [82]:
##Data ingestion -- from the website
from langchain_community.document_loaders import WebBaseLoader



In [83]:
loader = WebBaseLoader("https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html")
loader

In [84]:
docs = loader.load()
print(docs)

[Document(metadata={'source': 'https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html', 'title': 'ChatPromptTemplate — 🦜🔗 LangChain  documentation', 'language': 'en'}, page_content='\n\n\n\n\n\n\n\n\nChatPromptTemplate — 🦜🔗 LangChain  documentation\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to main content\n\n\nBack to top\n\n\n\n\nCtrl+K\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n    Reference\n  \n\n\n\n\n\n\n\n\n\nCtrl+K\n\n\n\n\n\n\n\nDocs\n\n\n\n\n\n\n\n\n\n\nGitHub\n\n\n\nX / Twitter\n\n\n\n\n\n\n\n\nCtrl+K\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n    Reference\n  \n\n\n\n\n\n\n\n\n\n\nDocs\n\n\n\n\n\n\n\n\n\n\nGitHub\n\n\n\nX / Twitter\n\n\n\n\n\n\n\nSection Navigation\nBase packages\n\nCore\nagents\nbeta\ncaches\ncallbacks\nchat_history\nchat_loaders\nchat_sessions\ndocument_loaders\ndocuments\nembeddings\nexample_selectors\nexceptions\nglobals\nindexing\nlanguage_models\nload\nmessages\nout

In [85]:
### Load data -->then divide to chunks--->embedding to vector from text-->then store in vector embedding
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(docs)


In [86]:
documents

[Document(metadata={'source': 'https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html', 'title': 'ChatPromptTemplate — 🦜🔗 LangChain  documentation', 'language': 'en'}, page_content='ChatPromptTemplate — 🦜🔗 LangChain  documentation\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to main content\n\n\nBack to top\n\n\n\n\nCtrl+K\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n    Reference\n  \n\n\n\n\n\n\n\n\n\nCtrl+K\n\n\n\n\n\n\n\nDocs\n\n\n\n\n\n\n\n\n\n\nGitHub\n\n\n\nX / Twitter\n\n\n\n\n\n\n\n\nCtrl+K\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n    Reference\n  \n\n\n\n\n\n\n\n\n\n\nDocs\n\n\n\n\n\n\n\n\n\n\nGitHub\n\n\n\nX / Twitter\n\n\n\n\n\n\n\nSection Navigation\nBase packages'),
 Document(metadata={'source': 'https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html', 'title': 'ChatPromptTemplate — 🦜🔗 LangChain  documentation', 'language': 'en'}, page_c

In [87]:
# Embed and store in vector DB
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(api_key=api_key2, model="text-embedding-3-large")

In [88]:
from langchain_community.vectorstores import FAISS
vectorstoredb = FAISS.from_documents(documents, embeddings)

In [89]:
vectorstoredb

In [90]:
##direct query to vector db 
query = "Below is a table that illustrates some events that might be emitted by various chains. Metadata fields have been omitted from the table for brevity. Chain definitions have been included after the table."
result = vectorstoredb.similarity_search(query)

In [91]:
print(result[0].page_content)

tags: Optional[List[str]] - The tags of the Runnable that generatedthe event.




metadata: Optional[Dict[str, Any]] - The metadata of the Runnablethat generated the event.



data: Dict[str, Any]

Below is a table that illustrates some events that might be emitted by various
chains. Metadata fields have been omitted from the table for brevity.
Chain definitions have been included after the table.
ATTENTION This reference table is for the V2 version of the schema.


event
name
chunk
input
output



on_chat_model_start
[model name]

{“messages”: [[SystemMessage, HumanMessage]]}


on_chat_model_stream
[model name]
AIMessageChunk(content=”hello”)



on_chat_model_end
[model name]

{“messages”: [[SystemMessage, HumanMessage]]}
AIMessageChunk(content=”hello world”)

on_llm_start
[model name]

{‘input’: ‘hello’}


on_llm_stream
[model name]
‘Hello’



on_llm_end
[model name]

‘Hello human!’


on_chain_start
format_docs




on_chain_stream
format_docs
“hello world!, goodbye world!”


In [92]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(api_key=api_key2, model="gpt-4-turbo-preview")

In [93]:
### Retrievel Chain , document chain
# Create prompt with correct context and input placeholders
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(
    """Answer the question based only on the following context:
{context}

Question: {input}"""
)

### We can use retriever to dynamically select the most relevent documents and pass those in for a given question.

In [94]:
# Build document chain
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm, prompt)

# Create retrieval chain
retriever = vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
rag_chain = create_retrieval_chain(retriever, document_chain)

In [95]:
# Invoke the chain with the correct input format
query = "Below is a table that illustrates some events that might be emitted by various chains. Metadata fields have been omitted from the table for brevity. Chain definitions have been included after the table."
response = rag_chain.invoke({"input": query})
print(response["answer"])

Based on the provided context, the table illustrates the variety of events that can be emitted by various chains within a given system, categorized by the distinct stages or actions occurring within those chains, such as starting, streaming, and ending. These events are classified by their "event name" and are associated with specific "input" and "output" data relevant to the operation or processing step. The events span multiple functionalities and technologies, including chat models, language learning models (LLMs), document formatting, tool executions, retrieval operations, and prompt handling. The table serves as a reference for understanding how these events function within the V2 version of the schema, detailing the flow of data and the types of interactions that can occur within the system's ecosystem.

To address the enumerated topics:

1. **How to chain runnables**: Chaining runnables involves sequentially connecting the output of one runnable as the input to another, creating